In [1]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [2]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

In [3]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [4]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "深海蓝藻保湿面膜" in product_name:
        return "深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。"
    elif "美白精华" in product_name:
        return "美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": mock_query_product_database,
    "generate_emoji": mock_generate_emoji,
}

In [6]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。

    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。

    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """

    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")

    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]

    iteration_count = 0
    final_response = None

    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")

        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION, # 告知模型可用的工具
                tool_choice="auto" # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message

            # **ReAct模式：处理工具调用**
            if response_message.tool_calls: # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message) # 将工具调用信息添加到对话历史

                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")

                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result) # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs) # 将工具执行结果作为 Observation 添加到对话历史

            # **ReAct 模式：处理最终内容**
            elif response_message.content: # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")

                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)

                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break

        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break

    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

In [7]:
# 测试案例 1: 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '深海蓝藻保湿面膜'}
[Tool Call] 模拟查询产品数据库：深海蓝藻保湿面膜
Observation: 工具返回结果：深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '2024年小红书面膜爆款文案趋势 保湿面膜流行元素'}
[Tool Call] 模拟搜索网页：2024年小红书面膜爆款文案趋势 保湿面膜流行元素
Observation: 工具返回结果：小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '惊喜补水效果 水光肌 深层保湿 肌肤焕新'}
[Tool Call] 模拟生成表情符号，上下文：惊喜补水效果 水光肌 深层保湿 肌肤焕新
Observation: 工具返回结果：['💦', '💧', '🌊', '✨']
-- Iteration 4 --
[模型生成结果] **Observation**: 现在我已经收集到足够的信息：
- 产品特点：深海蓝藻提取物、深层补水、修护屏障、舒缓敏感
- 用户痛点：卡粉、泛红、紧绷感
- 流行趋势：沙漠干皮救星、熬夜急救、水光肌养成
- 表情符号：💦💧🌊✨

现在我将创作一篇活泼甜美的小红书爆款文案。

```json
{
  "title": "沙漠干皮救星！深海蓝藻面膜让我一夜变水光肌✨",
  "body": "姐妹们！挖到宝了！！这个深海蓝藻保湿面膜真的绝了💦\n\n作为一个资深干皮党，每到换季就卡粉到怀疑人生😭 直到遇见它！\n\n🌊深海蓝藻提

In [8]:
product_name_2 = "美白精华"
tone_style_2 = "知性温柔"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '美白精华小红书爆款文案 2024最新趋势 用户评价'}
[Tool Call] 模拟搜索网页：美白精华小红书爆款文案 2024最新趋势 用户评价
Observation: 工具返回结果：未找到关于 '美白精华小红书爆款文案 2024最新趋势 用户评价' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '美白精华'}
[Tool Call] 模拟查询产品数据库：美白精华
Observation: 工具返回结果：美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '美白提亮 温柔知性 护肤效果'}
[Tool Call] 模拟生成表情符号，上下文：美白提亮 温柔知性 护肤效果
Observation: 工具返回结果：['🌿', '💯', '👍']
-- Iteration 4 --
[模型生成结果] **Observation**: 基于收集到的信息，我了解到：
- 美白精华的核心成分是烟酰胺和VC衍生物
- 主要功效包括提亮肤色、淡化痘印、改善暗沉
- 质地轻薄易吸收
- 适合需要均匀肤色的人群
- 表情符号需要更多美白护肤相关的

现在我可以创作一篇知性温柔风格的小红书爆款文案了。

```json
{
  "title": "温柔白月光养成记｜这支美白精华真的让我发光了✨",
  "body": "姐妹们，今天来分享一支让我从暗沉肌逆袭成白月光的神仙精华💫\n\n🌿核心成分：烟酰胺+VC衍生物\n这对黄金搭档真的绝了！烟酰胺温和提亮，VC衍生物稳定抗氧化，双管齐下让肤色均匀透亮\n\n✨使用感受：\n

In [16]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", [])

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题

    # 正文，保留换行符
    markdown_output += f"{body}\n\n"

    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"

    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"

    return markdown_output.strip() # 去除末尾多余的空白

In [11]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟",
  "body": "姐妹们！我终于找到了我的本命美白精华！💖\\n\\n作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩\\n\\n🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！\\n💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！\\n🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！\\n\\n使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～\\n\\n真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤好物", "#冷白皮"],
  "emojis": ["✨", "💖", "🤩", "💧", "🌿"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)

--- 格式化后的小红书文案 (Markdown) ---
## ✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟

姐妹们！我终于找到了我的本命美白精华！💖

作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩

🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！
💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！
🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！

使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～

真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖

#美白精华 #提亮肤色 #淡化痘印 #护肤好物 #冷白皮

--- 格式化错误示例 ---
错误：无法解析 JSON 字符串 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
原始字符串：
{'title': 'Test', 'body': 'This is not valid json'}


In [12]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_1)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 沙漠干皮救星！深海蓝藻面膜让我一夜变水光肌✨

姐妹们！挖到宝了！！这个深海蓝藻保湿面膜真的绝了💦

作为一个资深干皮党，每到换季就卡粉到怀疑人生😭 直到遇见它！

🌊深海蓝藻提取物真的不是盖的
敷完瞬间感觉皮肤在喝水💧
第二天上妆完全不卡粉
连闺蜜都问我是不是偷偷打了水光针

最惊喜的是敏感泛红也舒缓了很多
质地清爽不粘腻 敷完直接睡觉都不怕
熬夜党必备的急救神器✨

已经安利给身边所有干皮姐妹
都说用完皮肤水当当的
性价比超高 学生党也能冲！

#干皮救星 #熬夜党必备

#深海蓝藻面膜 #保湿面膜推荐 #干皮救星 #水光肌养成 #敏感肌友好


In [20]:
product_name_3 = "深海蓝藻保湿面膜"
tone_style_3 = "幽默风趣"
result_3 = generate_rednote(product_name_3, tone_style_3)

print("\n--- 生成的文案 3 ---")
print(result_3)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：幽默风趣

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '深海蓝藻保湿面膜'}
[Tool Call] 模拟查询产品数据库：深海蓝藻保湿面膜
Observation: 工具返回结果：深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '小红书 保湿面膜 爆款文案 2024 流行趋势'}
[Tool Call] 模拟搜索网页：小红书 保湿面膜 爆款文案 2024 流行趋势
Observation: 工具返回结果：小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '深海蓝藻面膜 用户评价 效果反馈'}
[Tool Call] 模拟搜索网页：深海蓝藻面膜 用户评价 效果反馈
Observation: 工具返回结果：未找到关于 '深海蓝藻面膜 用户评价 效果反馈' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。
-- Iteration 4 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '深海蓝藻保湿面膜 补水效果 惊喜 幽默风趣'}
[Tool Call] 模拟生成表情符号，上下文：深海蓝藻保湿面膜 补水效果 惊喜 幽默风趣
Observation: 工具返回结果：['💦', '💧', '🌊', '✨']
-- Iteration 5 --
[模型生成结果] **Observation**: 我已经收集到足够的信息：
- 产品核心卖点：深海蓝藻提取物、深层补水、修护屏障、舒缓

In [21]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_3)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 沙漠干皮救星！深海蓝藻面膜让我从干涸到水光肌💦

姐妹们！我宣布这是我用过最绝的保湿面膜！

💧 作为一个常年卡粉的沙漠干皮，每次化妆都像在脸上开荒种地...直到遇见了深海蓝藻面膜！

🌊 这个面膜真的绝了！深海蓝藻提取物就像给皮肤喝了一整片海洋，敷完瞬间水润到发光！

✨ 质地清爽不粘腻，敷完不用洗直接睡觉，第二天起床皮肤嫩得像剥了壳的鸡蛋！

💖 最惊喜的是，泛红敏感也改善了！现在化妆再也不卡粉了，男朋友都说我皮肤变好了～

🔥 熬夜党必备！敷一片第二天照样水光肌，谁用谁知道！

#深海蓝藻面膜 #保湿面膜推荐 #沙漠干皮救星 #水光肌养成 #熬夜急救面膜


In [22]:
from glob import glob

mfd_text_lines = []

for file_path in glob("mm.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    mfd_text_lines += file_text.split("# ")

In [23]:
len(mfd_text_lines)

1

In [25]:
from pymilvus import MilvusClient

mm_milvus_client = MilvusClient(uri="./mm_milvus_demo.db")

mm_collection_name = "my_mm_rag_collection"

In [26]:
if mm_milvus_client.has_collection(mm_collection_name):
    mm_milvus_client.drop_collection(mm_collection_name)

In [27]:
from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key='sk-xsV0B5iUhLxrQJMT0a64Be7939D940Eb8f71A566E4B0052a', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

/Users/allenguo/data/workspace/deepseek-quickstart/.venv1/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)

512


In [30]:
mm_milvus_client.create_collection(
    collection_name=mm_collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [31]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(mfd_text_lines)

for i, line in enumerate(tqdm(mfd_text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

mm_milvus_client.insert(collection_name=mm_collection_name, data=data)

Creating embeddings: 100%|██████████| 1/1 [00:00<00:00, 12826.62it/s]


{'insert_count': 1, 'ids': [0], 'cost': 0}

In [35]:
import json
def query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 实际查询产品数据库：{product_name}")
    search_res = mm_milvus_client.search(
        collection_name=mm_collection_name,
        data=embedding_model.encode_queries(
            [product_name]
        ),  # 将问题转换为嵌入向量
        limit=3,  # 返回前3个结果
        search_params={"metric_type": "IP", "params": {}},  # 内积距离
        output_fields=["text"],  # 返回 text 字段
    )
    retrieved_lines_with_distances = [
        (res["entity"]["text"], res["distance"]) for res in search_res[0]
    ]
    context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
    )
    print("查询向量数据库结果")
    print(context)
    return context;

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": query_product_database,
    "generate_emoji": mock_generate_emoji,
}

In [36]:
product_name_4 = "面膜核心成分与功效"
tone_style_4 = "形象生动"
result_4 = generate_rednote(product_name_4, tone_style_4)

print("\n--- 生成的文案 4 ---")
print(result_4)


🚀 启动小红书文案生成助手，产品：面膜核心成分与功效，风格：形象生动

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '面膜核心成分与功效'}
[Tool Call] 实际查询产品数据库：面膜核心成分与功效
查询向量数据库结果
**
面膜类型
贴片式面膜
描述: 将精华液吸附在面膜纸或无纺布上，覆盖面部肌肤，通过短暂封闭促进吸收。
特点: 方便卫生，精华液含量高，见效快，常用于强效补水和急救。
适用肤质: 几乎所有肤质，可根据精华液成分进行具体选择。
使用频率: 每周2-3次。

膏状/泥状面膜
描述: 包括清洁泥膜、睡眠面膜、保湿啫喱等，需手动涂抹，敷一定时间后洗掉或保留。
特点: 清洁型（泥膜）吸附力强；滋养型（睡眠面膜）渗透性佳。
适用肤质: 清洁型适合油性、混合性肌；滋养型适合干性、熟龄肌。
使用频率: 清洁型每周1-2次；滋养型每周1-2次。

剥离式面膜
描述: 通常为胶状液体，涂在脸上后会形成一层薄膜，干燥后可从脸上撕下。
特点: 能带去老化角质和黑头，但撕拉过程可能对皮肤有刺激。
适用肤质: 油性、耐受性肌肤，敏感肌和痘痘肌慎用。
使用频率: 每周不超过1次。

凝胶状面膜
描述: 通常是透明或半透明的凝胶质地，主打镇静、舒缓。
特点: 肤感清凉，保湿效果好，常用于晒后修复或敏感肌舒缓。
适用肤质: 敏感肌、油性肌，或日晒后需要镇静的肌肤。
使用频率: 按需使用，可每天一次。

**
面膜核心成分与功效
补水保湿成分
透明质酸(玻尿酸): 强力抓水，能吸收自身重量数百倍的水分，深层补水。
甘油: 经典的保湿剂，通过在皮肤表面形成薄膜来防止水分蒸发。
神经酰胺: 皮肤角质层中固有的脂质，能修复皮肤屏障，从而锁住水分。
泛醇(维生素B5): 具有强效保湿和修复效果，能舒缓肌肤。

美白提亮成分
烟酰胺: 能够抑制黑色素向皮肤表层的转移，从而实现提亮肤色、淡化痘印的效果。
维生素C及其衍生物: 具有强大的抗氧化能力，能抑制黑色素的生成，淡化已形成的斑点。
熊果苷: 一种温和的酪氨酸酶抑制剂，能有效淡化色斑。
传明酸: 对于改善肤色不均和肝斑有较好效果。

抗老紧致成分
视黄醇(A醇): 抗老的黄金标准成分

In [37]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_4)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 面膜成分大揭秘！选对成分=选对效果💫

姐妹们！今天来聊聊面膜的核心成分，选对面膜真的能让护肤效果翻倍！✨

💦补水保湿天团：
• 透明质酸：吸水力超强，能吸收自身数百倍水分
• 神经酰胺：修复皮肤屏障，锁水小能手
• 甘油：经典保湿剂，防止水分蒸发

✨美白提亮战队：
• 烟酰胺：抑制黑色素转移，淡化痘印
• 维C衍生物：抗氧化王者，提亮肤色
• 熊果苷：温和淡斑，改善暗沉

🌟抗老紧致精英：
• 视黄醇：抗老黄金标准，促进胶原蛋白
• 胜肽：抚平细纹，紧致肌肤
• 玻色因：提升皮肤弹性

🌿舒缓修护专家：
• 积雪草：修复镇静，敏感肌救星
• 角鲨烷：亲和力强，修复屏障

选面膜小贴士：
✅油皮：选清洁控油成分（水杨酸、活性炭）
✅干皮：选高保湿成分（透明质酸、神经酰胺）
✅敏感肌：选舒缓成分（积雪草、尿囊素）
✅痘痘肌：选抗炎成分（茶树精油、壬二酸）

记住：成分决定效果！选对成分，护肤事半功倍～💖

#面膜成分 #护肤干货 #面膜选择指南 #成分党 #护肤科普
